In [1]:
# Importing libraries 
import numpy as np
import pandas as pd
import ast

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Loading output of GA algorithms from file
file_path = "output/results.csv"

results_df = pd.read_csv(file_path)
results_df.head(6)

,run_number,total_runs,pop_size,gens,select,crossover,mutate,xo_prob,mut_prob,tourn_size,best_fitness_lst
0,1,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,4.0,"[0.520966649055481, 0.8256333470344543, 0.8726..."
1,2,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,4.0,"[0.5569666624069214, 0.5575000047683716, 0.569..."
2,3,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,4.0,"[0.3711666762828827, 0.5481166839599609, 0.567..."
3,4,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,4.0,"[0.8169833421707153, 0.8779833316802979, 0.877..."
4,1,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,2.0,"[0.5535500049591064, 0.553683340549469, 0.7488..."
5,2,4,20,5,tournament_sel,arithmetic_xo,arithmetic_mutation,0.9,0.1,2.0,"[0.6256999969482422, 0.8153333067893982, 0.867..."


In [3]:
# Filling NaN values for 'torun_size' with "n/a" (will be useful for further grouping)
results_df = results_df.fillna({'tourn_size': 'n/a'})

# Convert string representation of arrays to actual arrays
results_df['best_fitness_lst'] = results_df['best_fitness_lst'].str.strip('[]').str.split(',').apply(lambda x: [float(val) for val in x])

In [5]:
# Grouping runs of a certain configuration
configuration_columns = ["total_runs", "pop_size", "gens", "select", "crossover", "mutate", 
                         "xo_prob", "mut_prob", "tourn_size"]
grouped = results_df.groupby(configuration_columns)

# Group by the specified columns and calculate the mean values of arrays
grouped = results_df.groupby(configuration_columns)['best_fitness_lst'].apply(lambda x: np.mean(x.tolist(), axis=0))

# Reset the index to convert the resulting Series back to a DataFrame
result = grouped.reset_index()
result = result.rename(columns={'best_fitness_lst': 'ABF'})

print(result)

   total_runs  pop_size  gens          select        crossover  \
0           4        20     5             fps    arithmetic_xo   
1           4        20     5  tournament_sel    arithmetic_xo   
2           4        20     5  tournament_sel    arithmetic_xo   
3           4        20     5  tournament_sel  single_point_co   

                mutate  xo_prob  mut_prob tourn_size  \
0  arithmetic_mutation      0.9       0.1        n/a   
1  arithmetic_mutation      0.9       0.1        2.0   
2  arithmetic_mutation      0.9       0.1        4.0   
3   inversion_mutation      0.9       0.1        4.0   

                                                 ABF  
0  [0.7625875025987625, 0.8135041743516922, 0.839...  
1  [0.5116500109434128, 0.5734666585922241, 0.650...  
2  [0.5665208324790001, 0.7023083418607712, 0.721...  
3  [0.6482708305120468, 0.7366458177566528, 0.766...  
